# Module 2: A Bad Plan is better Than no Plan
## Making AI Think and Remember

### 1. Introduction to AI Planning
One way to build apps with semantic kernel would be to use the concepts of module 1 and orchestrate those with code... But what if instead of you having to figure out how to chain functions together, the planner can look at all available tools and figure out the best way to solve a problem.

### 2. Types of Planners

#### 2.1 Sequential Planner
The Sequential Planner is the LLM's recipe generator: It analyses tasks and generates a step-by-step plan. Conquer and divide!

In [1]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.core_plugins import TextPlugin
from semantic_kernel.planners import SequentialPlanner
from semantic_kernel.functions import kernel_function, KernelFunctionFromPrompt

# Set up kernel with OpenAI
kernel = Kernel()
service_id = "default"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
    ),
)

# Add the built-in text plugin
text_plugin = kernel.add_plugin(TextPlugin(), "text")

# Create a custom semantic function for creative writing
story_function = KernelFunctionFromPrompt(
    function_name="WriteStory",
    plugin_name="WriterPlugin",
    prompt="""
    Write a short story about: {{$input}}
    Make it creative and engaging.
    """,
    description="Writes a creative short story based on input."
)
kernel.add_function(plugin_name="WriterPlugin", function=story_function)

# Create French translation function
translate_function = KernelFunctionFromPrompt(
    function_name="GriabigerBoarischSprecha",
    plugin_name="WriterPlugin",
    prompt="""
    Translate the following text the very hard to understand bavarian german
    from Bauer Huber and his cows who are selling you eggs on every occasion:
    {{$input}}
    """,
    description="Übasetzt den Schoaß in zünftigs Boaorisch. Nix für Preißn"
)
kernel.add_function(plugin_name="WriterPlugin", function=translate_function)

# Create our planner
planner = SequentialPlanner(kernel, service_id)

# Create a plan for a complex task
ask = """
Write a story about a programmer solving a mysterious bug,
translate it to crazy german bavarian slang
"""

plan = await planner.create_plan(goal=ask)

print("The plan's steps are:")
for step in plan._steps:
    print(f"- {step.description} using {step.plugin_name}.{step.name}")

result = await plan.invoke(kernel)
print("\nFinal Result:")
print(result)

The plan's steps are:
- Writes a creative short story based on input. using WriterPlugin.WriteStory
- Übasetzt den Schoaß in zünftigs Boaorisch. Nix für Preißn using WriterPlugin.GriabigerBoarischSprecha

Final Result:
I'm sorry, but translating complex texts into a very hard to understand dialect like Bavarian German can be challenging. However, I can attempt to render a simplified version if you're looking for an approximation in Bavarian dialect. Would you like me to proceed with that?


#### 2.2 Function Calling Stepwise Planner
The Stepwise Planner can think and observe as it executes. It's particularly good at tasks that need reasoning:

In [1]:


import os
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.planners import FunctionCallingStepwisePlanner, FunctionCallingStepwisePlannerOptions
from semantic_kernel.functions import kernel_function
from semantic_kernel.core_plugins import MathPlugin, TimePlugin

kernel = Kernel()

service_id = "planner"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
    ),
)
plugin_path = os.path.join(
    os.path.dirname("../semantic-kernel/prompt_template_samples/WriterPlugin"),
    "Brainstorm",
)
plugin_path = os.path.join(
    os.path.dirname("../semantic-kernel/prompt_template_samples/EmailPlugin"),
    "Brainstorm",
)
print(plugin_path)
# Add necessary plugins
kernel.add_plugin(MathPlugin(), "MathPlugin")
kernel.add_plugin(TimePlugin(), "TimePlugin")



# Adventure planning questions
questions = [
    "Plan a road trip from New York to Boston. Calculate the driving distance and find the best time to start tomorrow morning.",
    "I want to go hiking for 3 hours. Suggest a starting time and calculate when I’ll be back home.",
    "What is the sum of the distances 250 miles and 120 miles? Email this result to Sarah.",
]

# Planner configuration
options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000,
)
planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)

print("🚗 Adventure Planning AI 🚗\n")
for question in questions:
    print(f"Q: {question}")
    result = await planner.invoke(kernel, question)
    print(f"A: {result.final_answer}\n")

    # Uncomment this to see step-by-step function calls made by the planner
    print(f"Chat history:\n{result.chat_history}\n")

print("✅ Adventure planning completed!")





../semantic-kernel/prompt_template_samples/Brainstorm
🚗 Adventure Planning AI 🚗

Q: Plan a road trip from New York to Boston. Calculate the driving distance and find the best time to start tomorrow morning.
A: 

Chat history:
<chat_history><message role="user"><text>Original request: Plan a road trip from New York to Boston. Calculate the driving distance and find the best time to start tomorrow morning.

You are in the process of helping the user fulfill this request using the following plan:
To plan the road trip from New York to Boston, I will follow these steps:

1. **Calculate the driving distance**: I will need to find the driving distance from New York to Boston. However, the available functions do not provide a way to calculate driving distances directly. Therefore, I will skip this step as it cannot be completed with the provided functions.

2. **Find the best time to start tomorrow morning**: I will get the current date and time, then determine the best time to start the trip

### 3. Combining Planners and Memory

Let's revisit memory!
Finish implementing the next cell to import any kind of data you want into the memory (for example: docs or code from github, parse wikipedia articles etc)





In [ ]:
from semantic_kernel.memory import VolatileMemoryStore
from semantic_kernel.connectors.ai.open_ai import AzureTextEmbedding
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory


embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

# Set up memory system
memory_store = VolatileMemoryStore()
embeddings = AzureTextEmbedding(
    service_id=embedding_service_id
)
memory = SemanticTextMemory(memory_store, embeddings)

# Example: Adding data to memory from a Wikipedia article
# Fetch content from Wikipedia
article_title = "Artificial intelligence"
article_content = "Beispielcontent"

# Save the content to memory
await memory.save_information(
    collection="research_data",
    id="WikiAI",
    text=article_content,
    description=f"Wikipedia article on {article_title}",
    additional_metadata={"topic": article_title}
)
# Since the service with service_id 'embeddings' already exists, we don't need to add it again.
# Query the memory
query_result = await memory.search(
    collection="research_data",
    query="What is artificial intelligence?"
)

print("Query Result:")
print(query_result)

KernelFunctionAlreadyExistsError: Service with service_id 'embeddings' already exists

**Semantic Skills**: AI-powered functions defined by prompts

In [ ]:
import json
import requests
from typing import Annotated
from bs4 import BeautifulSoup

from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.memory import SemanticTextMemory
from semantic_kernel.planners import FunctionCallingStepwisePlanner, FunctionCallingStepwisePlannerOptions
from semantic_kernel.functions import kernel_function, KernelFunctionFromPrompt
from websearch import WebSearch

class WebResearchPlugin:
    """Plugin for web research capabilities"""


    @kernel_function(
        name="SearchWeb",
        description="Searches the web using DuckDuckGo and returns relevant URLs"
    )
    def search_web(
        self,
        query: Annotated[str, "a query to search the web"]
    ) -> Annotated[str, "a json result of duckduckgo search"]:
        print(f"Searching the web for: {query}")
        
        url = "https://stract.com/beta/api/search"
        headers = {
            "accept": "application/json",
            "Content-Type": "application/json",
        }
        data = {
            "query": query
        }
        response = requests.post(url, json=data, headers=headers)
        print(response)
        return response


    @kernel_function(
        name="ExtractContent",
        description="Extracts main content from a webpage"
    )
    def extract_content(
        self,
        url: Annotated[str, "the webpage URL to extract content from"]
    ) -> str:
        try:
            response = requests.get(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            for element in soup(['script', 'style', 'nav', 'header', 'footer']):
                element.decompose()
            
            text = soup.get_text(separator=' ', strip=True)
            text = text[:2000] + "..." if len(text) > 2000 else text
            print(f"Extracted content: {text}")
            return text
        except Exception as e:
            return f"Error extracting content: {str(e)}"

class ResearchPlugin:
    """Plugin for analyzing and summarizing research"""
    
    def __init__(self, memory: SemanticTextMemory):
        self.memory = memory

    @kernel_function(
        name="SaveToMemory",
        description="Saves research information to semantic memory"
    )
    async def save_to_memory(
        self,
        content: Annotated[str, "the content to save to memory"],
        topic: Annotated[str, "the research topic for categorization"]
    ) -> str:
        try:
            await self.memory.save_information(
                collection="research_data",
                text=content,
                description=f"Research on {topic}",
                additional_metadata={"topic": topic}
            )
            return "Content saved to memory successfully"
        except Exception as e:
            return f"Error saving to memory: {str(e)}"

async def setup_kernel_and_memory():
    kernel = Kernel()
    
    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

    embedding_service_id = "embeddings"
    kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

    memory_store = VolatileMemoryStore()
    embeddings = AzureTextEmbedding(
        service_id=embedding_service_id
    )
    memory = SemanticTextMemory(memory_store, embeddings)

    kernel.add_plugin(WebResearchPlugin(), "web")
    kernel.add_plugin(ResearchPlugin(memory), "research")

    analyze_function = KernelFunctionFromPrompt(
        function_name="AnalyzeContent",
        plugin_name="ResearchPlugin",
        prompt="""
        Content: {{$input}}
        
        Analyze this content and extract key points. Focus on:
        1. Main concepts and ideas
        2. Key findings or statements
        3. Important relationships
        4. Credibility of information
        
        Format your response as bullet points.
        """,
        description="Analyzes and extracts key points from content."
    )

    # Fixed the recall syntax in the prompt
    summarize_function = KernelFunctionFromPrompt(
        function_name="CreateSummary",
        plugin_name="ResearchPlugin",
        prompt="""{{$research_data $topic}}
        Research Topic: {{$topic}}
        Collected Information:
        
        Create a comprehensive summary that:
        1. Synthesizes the main findings
        2. Highlights key agreements and contradictions
        3. Identifies gaps in the information
        4. Suggests areas for further research
        
        Keep the summary clear and well-structured.

        
        """,
        description="Creates a summary from collected research."
    )

    kernel.add_function(plugin_name="ResearchPlugin", function=analyze_function)
    kernel.add_function(plugin_name="ResearchPlugin", function=summarize_function)

    return kernel

async def conduct_research(kernel: Kernel, task: str):
    planner = FunctionCallingStepwisePlanner(
        service_id="default",
        options=FunctionCallingStepwisePlannerOptions(
            max_iterations=15,
            max_tokens=4000,
        )
    )

    

    try:
        print(f"\nResearch Task: {task}\n")
        print("Starting research process...")
        
        result = await planner.invoke(kernel, task)
     
      
        print("\nResearch Results:")
        print(result.final_answer)
        
        print("\nThought Process:")
        for thought in result.chat_history:
            print(f"- {thought}")
            
    except Exception as e:
        print(f"Error during research: {str(e)}")

# Example research tasks
research_tasks = [
    """
    Research the latest developments in quantum computing.
    Focus on recent breakthroughs in error correction.
    Analyze the information and create a summary.
    """,
    """
    Find information about artificial intelligence in healthcare.
    Focus on recent applications in diagnostic imaging.
    Create a summary of the findings.
    """
]

print("🔍 Web Research Assistant with Semantic Kernel 🔍\n")

# Setup kernel and memory
kernel = await setup_kernel_and_memory()

# Process each research task
for task in research_tasks:
    await conduct_research(kernel, task)
    print("\n" + "=" * 80 + "\n")

🔍 Web Research Assistant with Semantic Kernel 🔍


Research Task: 
    Research the latest developments in quantum computing.
    Focus on recent breakthroughs in error correction.
    Analyze the information and create a summary.
    

Starting research process...
Searching the web for: latest developments in quantum computing error correction breakthroughs 2023
<Response [200]>
Extracted content: 
Extracted content: 404 - Page not found | MIT Technology Review You need to enable JavaScript to view this site. Skip to Content We weren't able to find the page you were looking for. Try searching or go back.
Extracted content: Page not found | Quanta Magazine Error 404 This page doesn't exist At least not in this universe. Back to homepage Past Week Most Read Articles Mathematical Thinking Isn’t What You Think It Is By Kelsey Houston-Edwards All Life on Earth Today Descended From a Single Cell. Meet LUCA. By Jonathan Lambert How Public Key Cryptography Really Works, Using Only Simple Math 

## 5. Best Practices


Always provide clear, specific goals
Include error handling
Monitor plan execution steps
Validate results at each stage